## Basic usage
Initialize an instance of Daemon, then call update to scan the cwd and build the Knowledge Graph.

In [1]:
from pathlib import Path
from ragdaemon.daemon import Daemon

daemon = Daemon(Path.cwd())
await daemon.update()
print(len(daemon.graph.nodes), "Nodes")

279 Nodes


## Search
Return be files, chunks of files (function, class, method or 'BASE'), or diffs (edits to a particular path and line range) sorted by embedding similarity.

Each search result includes (among other things):
1. `id`: Used as the id for nodes in the graph.
    - For files and chunks, a relative path to cwd. e.g. `mentat/config.py:Config`.
    - For diffs, its target location, e.g. `mentat/config.py:10-15`. 
2. `distance`: how related to query (lower is better)
3. `ref`: path + line numbers, used to load the document.
    - For files and chunks, relative path + lines, e.g. `mentat/config.py:10-15`
    - For diffs, the diff target ("DEFAULT" if none provided) + lines in diff, e.g. `DEFAULT:4-10`
4. `document`: The embedded content. Always f"{id}\n{content}" so the path / filename is also embedded, and there are no duplicates.
5. `checksum`: an md5 hash of the document, used as the chroma index.

In [2]:
results = daemon.search("raycaster", n=5)
for i, result in enumerate(results):
    if i == 0:
        fields = set(result.keys())
        print(f"Fields: {fields}")
    print(f"{i+1}. {result['id']}")

print("\nExample Result:")
for k, v in results[2].items():
    print(f"---{k}---\n{v}")

Fields: {'ref', 'active', 'distance', 'id', 'checksum', 'document', 'type'}
1. ragdaemon/static/js/three/raycaster.js:BASE
2. ragdaemon/static/js/three/raycaster.js
3. ragdaemon/static/js/three/raycaster.js:getClickTarget
4. ragdaemon/static/js/three/renderer.js
5. ragdaemon/static/js/main.js:BASE

Example Result:
---active---
True
---checksum---
70c16ff70b31128f393224c5922efb4a
---id---
ragdaemon/static/js/three/raycaster.js:getClickTarget
---ref---
ragdaemon/static/js/three/raycaster.js:10-28
---type---
chunk
---document---
ragdaemon/static/js/three/raycaster.js:10-28
function getClickTarget(event, dblclick=false) {
    // Calculate mouse position in normalized device coordinates
    // (-1 to +1) for both components
    mouse.x = (event.clientX / container.clientWidth) * 2 - 1;
    mouse.y = - (event.clientY / container.clientHeight) * 2 + 1;

    // Update the picking ray with the camera and mouse position
    raycaster.setFromCamera(mouse, camera);

    // Calculate objects inters

## Get Context
Return a `ContextBuilder` with the search results pre-loaded.

The `ContextBuilder` contains
1. Methods to 'include' individual nodes: `add_ref(file_or_chunk_ref)` and `add_diff(diff_id)`. 
2. A `.render()` method to produce a single string to send to an LLM
3. All the logic related to how to sort, group and consolidate.

You can
- Call `daemon.get_context(query, auto_tokens).render()` straightaway for normal RAG
- Call `daemon.get_context("")` to return an empty context and add items manually
- Pass a ContextBuilder to get_context using the 'context_builder=' to add search results to existing context.

In [3]:
context_builder = daemon.get_context("")
context_builder.add_ref("ragdaemon/app.py:BASE")
print(context_builder.render())

ragdaemon/app.py:BASE
1:import argparse
2:import asyncio
3:import socket
4:import webbrowser
5:from contextlib import asynccontextmanager
6:from pathlib import Path
7:from typing import Any
8:
9:import uvicorn
10:from fastapi import FastAPI, Request
11:from fastapi.responses import HTMLResponse
12:from fastapi.staticfiles import StaticFiles
13:from spice import Spice
14:from starlette.templating import Jinja2Templates
15:
16:from ragdaemon.daemon import Daemon
17:from ragdaemon.database import DEFAULT_EMBEDDING_MODEL
18:from ragdaemon.llm import DEFAULT_COMPLETION_MODEL
19:
20:# Load daemon with command line arguments and visualization annotators
21:parser = argparse.ArgumentParser(description="Start the ragdaemon server.")
22:parser.add_argument(
23:    "--refresh", "-r", action="store_true", help="Refresh active records."
24:)
25:parser.add_argument(
26:    "--chunk-extensions",
27:    "-c",
28:    nargs="*",
29:    help="List of file extensions to chunk, e.g., .py .js",
30:)
31:pars

In [4]:
context_builder.context

{'ragdaemon/app.py:BASE': {'lines': {1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79},
  'tags': set(),
  'document': 'ragdaemon/app.py:1-66,78-86,105-106,114-115\nimport argparse\nimport asyncio\nimport socket\nimport webbrowser\nfrom contextlib import asynccontextmanager\nfrom pathlib import Path\nfrom typing import Any\n\nimport uvicorn\nfrom fastapi import FastAPI, Request\nfrom fastapi.responses import HTMLResponse\nfrom fastapi.staticfiles import StaticFiles\nfrom spice import Spice\nfrom starlett